In [ ]:
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import json
from scipy.optimize import curve_fit
import numpy as np

matplotlib.rcParams['path.simplify'] = False

In [ ]:
mu, sigma = 0.5, 0.17
mu2, sigma2 = -0.5, 0.17
#data generation
np.random.seed(123)
data=np.concatenate((np.random.normal(mu, sigma, 5000), np.random.normal(mu2, sigma2, 2500)))
y,x,_=plt.hist(data, 100, alpha=.3, label='data')
x=(x[1:]+x[:-1])/2 # for len(x)==len(y)

#x, y inputs can be lists or 1D numpy arrays

def gauss(x, mu, sigma, A):
    return A*np.exp(-(x-mu)**2/2/sigma**2)

def bimodal(x, mu1, sigma1, A1, mu2, sigma2, A2):
    return gauss(x,mu1,sigma1,A1)+gauss(x,mu2,sigma2,A2)

expected = (mu, sigma, 250, mu2, sigma2, 125)
params, cov = curve_fit(bimodal, x, y, expected)
sigma=np.sqrt(np.diag(cov))
x_fit = np.linspace(x.min(), x.max(), 500)

In [ ]:
G = nx.erdos_renyi_graph(9, 0.3)
for e in G.edges():
    weight = 0
    # ensure we only give weights between -1 and 1 that are non zero
    while not (-1 <= weight <= 1) or weight == 0:
        weight = np.random.choice(data)
    G[e[0]][e[1]]['weight']  = weight

edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
nw_weights = tuple(x+1 for x in weights)
widths = tuple(abs(4*x) for x in weights)
pos = nx.spring_layout(G, k=1)
d = dict(G.degree)

In [ ]:
fig = plt.figure(figsize=(10, 5))
(ax1, ax2) = fig.subplots(1, 2)

ax1.hist(data, 100, alpha=.3, label='data')
ax1.plot(x_fit, bimodal(x_fit, *params), color='red', lw=1, label='model')
ax1.set_title("Bimodal Distribution")

nx.draw_networkx_nodes(G, pos, node_color='#918F8F', node_size=100, ax=ax2)
nx.draw_networkx_labels(G, pos, font_size=10, font_color='k', ax=ax2)
edge_collection = nx.draw_networkx_edges(G, pos, edgelist=edges, edge_color=weights, edge_cmap=plt.cm.PuOr, width=widths, edge_vmin=-1, edge_vmax=1, ax=ax2)
ax2.set_title("Co-occurrence Network")
plt.colorbar(edge_collection)
plt.box(False)
fig.savefig("4-co-occ-net.pdf", format="pdf", bbox_inches="tight")
fig.show()